# <font color='Blue'> Planejamento da Solução </font>

## Input - Entrada
### Problema de Negócio
1. Quais são as coisas que um potencial comprador de imóvel residencial considera antes de comprar uma casa?
(A localização, o tamanho da propriedade, a proximidade de escritórios, escolas, parques, restaurantes, hospitais ou preço ) 
2. Conjunto de dados que descrevem imóveis em Bengaluru.

## Output - Saída

1. Grafico(s) demostrando as features mais importantes para comprar um imóveis
2. Modelo para prever o preço das casas em Bengaluru.

## Task - Processos

1. Quais são as coisas que um potencial comprador de imóvel residencial considera antes de comprar uma casa?
* Limpar os dados
* Criar features
* Criar hipoteses de negocio em relação as features
* Fazer EDA validando / refutando as hipoteses de negocio.






# 0.0. Imports

In [3]:
import re
import pandas as pd
import numpy as np

from geopy.geocoders import Nominatim

from sklearn import model_selection as ms
from sklearn import preprocessing as pp
from sklearn import dummy
from sklearn import metrics
from sklearn import linear_model as lm
from sklearn import ensemble as en

## 0.1. Helper Function

In [345]:
def descriptive_statistics(num_attr):
    # Central Tendency: mean, median
    c1 = pd.DataFrame(num_attr.apply(np.mean))
    c2 = pd.DataFrame(num_attr.apply(np.median))

    # Dispension: min, max, range, std, skew, kurtosis
    d1 = pd.DataFrame(num_attr.apply(min))
    d2 = pd.DataFrame(num_attr.apply(max))
    d3 = pd.DataFrame(num_attr.apply(lambda x: x.max() - x.min()))
    d4 = pd.DataFrame(num_attr.apply(lambda x: x.std()))
    
    # Measures of Shape
    s1 = pd.DataFrame(num_attr.apply(lambda x: x.skew()))
    s2 = pd.DataFrame(num_attr.apply(lambda x: x.kurtosis()))

    # concat
    m = pd.concat([d1,d2,d3,c1,c2,d4,s1,s2], axis=1).reset_index()
    m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
    return m


## 0.2. Load Data

In [285]:
df_raw = pd.read_csv('../data/Bengaluru_House_Data.csv')

# 1.0. Data Description

In [286]:
df1 = df_raw.copy()

*  **Area_type** - describes the area 
*  **Availability** - when it can be possessed or when it is ready(categorical and time-series) 
*  **Location** - where it is located in Bengaluru 
*  **Price** - Value of the property in lakhs(INR) M
*  **Size** - in BHK or Bedroom (1-10 or more) 
*  **Society** - to which society it belongs 
*  **Total_sqft** - size of the property in sq.ft 
*  **Bath** - No. of bathrooms 
*  **Balcony** - No. of the balcony 

## 1.1. Data dimensions

In [287]:
print('Number of Rows: {}'.format(df1.shape[0]))
print('Number of Columns: {}'.format(df1.shape[1]))

Number of Rows: 13320
Number of Columns: 9


## 1.2. Data Types

In [288]:
df1.dtypes

area_type        object
availability     object
location         object
size             object
society          object
total_sqft       object
bath            float64
balcony         float64
price           float64
dtype: object

## 1.3. Check NA

In [289]:
df1.isna().mean() *100

area_type        0.000000
availability     0.000000
location         0.007508
size             0.120120
society         41.306306
total_sqft       0.000000
bath             0.548048
balcony          4.572072
price            0.000000
dtype: float64

## 1.4. Replace NA

In [290]:
# drop features
# remove: balcony, 4% ?
df1 = df1.dropna(subset=['size', 'location', 'bath', 'balcony'])

## 1.5. Change Dtypes

In [344]:
df1.dtypes

area_type        object
availability     object
location         object
size             object
society          object
total_sqft       object
bath            float64
balcony         float64
price           float64
dtype: object

## 1.6. Descriptive Statistics

In [346]:
num_att = df1.select_dtypes(include=['int64', 'float64'])
cat_att = df1.select_dtypes(exclude=['int64', 'float64'])

### 1.6.1. Numerical Attributes

In [347]:
descriptive_statistics(num_att)

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,bath,1.0,40.0,39.0,2.617309,2.0,1.226000,4.590497,85.455663
1,balcony,0.0,3.0,3.0,1.584343,2.0,0.817287,0.005966,-0.544247
2,price,8.0,2912.0,2904.0,106.060778,70.0,131.766089,7.875011,107.376164


**Note:**
1. bath: 40 banheiros? 

### 1.6.2. Categorical Attributes

In [348]:
cat_att.describe(include=['object'])

,area_type,availability,location,size,society,total_sqft
count,12710,12710,12710,12710,7496,12710
unique,4,78,1265,27,2592,1976
top,Super built-up Area,Ready To Move,Whitefield,2 BHK,GrrvaGr,1200
freq,8481,10077,514,5152,80,788


In [349]:
cat_att['area_type'].value_counts(normalize=True)

Super built-up  Area    0.667270
Built-up  Area          0.181747
Plot  Area              0.144532
Carpet  Area            0.006452
Name: area_type, dtype: float64

In [353]:
cat_att['availability'].value_counts(normalize=True).head(10)

Ready To Move    0.792840
18-Dec           0.022895
18-May           0.022187
18-Apr           0.020535
18-Aug           0.015736
19-Dec           0.014319
18-Jul           0.011015
18-Mar           0.009284
20-Dec           0.007710
18-Jun           0.007553
Name: availability, dtype: float64

In [354]:
cat_att['location'].value_counts(normalize=True).head(10)

Whitefield               0.040441
Sarjapur  Road           0.029268
Electronic City          0.023603
Kanakpura Road           0.020535
Thanisandra              0.018175
Yelahanka                0.016208
Uttarahalli              0.014634
Hebbal                   0.013611
Raja Rajeshwari Nagar    0.013218
Marathahalli             0.012903
Name: location, dtype: float64

In [355]:
cat_att['size'].value_counts(normalize=True).head(10)

2 BHK        0.405350
3 BHK        0.324784
4 Bedroom    0.058930
1 BHK        0.041699
3 Bedroom    0.041463
4 BHK        0.038474
2 Bedroom    0.025806
5 Bedroom    0.020692
6 Bedroom    0.013297
1 Bedroom    0.008261
Name: size, dtype: float64

In [364]:
dirt = df1.loc[~df1['total_sqft'].apply(lambda x: bool(re.search(   '^([0-9]+)$', x    ))), 'total_sqft']

len(dirt)

272

In [367]:
dirt.tolist()[:10]

['2100 - 2850',
 '1330.74',
 '3067 - 8156',
 '1042 - 1105',
 '1563.05',
 '1145 - 1340',
 '1015 - 1540',
 '2023.71',
 '1113.27',
 '34.46Sq. Meter']

**Note**:

1. availability: coluna categoricas 78 variavéis categoricas (Ready To Move 79 %)
2. location - capturar lat e long e deletar variavel.
3. size: Limpar variavel e transformar em inteira.
4. society: remover, pouca informação.
5. total_sqft: 272 itens que não são somente números.

# 2.0. Feature Engineering

In [370]:
# df2 = df1.copy()

## 2.1. Feature Creation

In [392]:
# %%time 

# location -> lat and lon

# geolocator = Nominatim(user_agent="geoapiExercises")

# def location_lat(x):
#     if geolocator.geocode(x, timeout=None):
#         return geolocator.geocode(x, timeout=None).raw['lat']
#     else: 
#         return x

# df2['lat'] = df2['location'].apply(location_lat)

# def location_lon(x):
#     if geolocator.geocode(x, timeout=None):
#         return geolocator.geocode(x, timeout=None).raw['lon']
#     else: 
#         return x

# df2['lon'] = df2['location'].apply(location_lon)

# df2.to_csv('bengaluru_house_data_lat_lon.csv', index=False)

df2 = pd.read_csv('../data/bengaluru_house_data_lat_lon.csv')

In [405]:
# size -> qt_bedroom

df2['qt_bedroom'] = df2['size'].apply(lambda x: str(x).split()[0]).astype(int)

# 3.0. Data Filtering

In [414]:
df3 = df2.copy()

## 3.1. Filter Rows

In [415]:
# lat and lon
df3_lat_dirt = df3.loc[~df3['lat'].apply(lambda x: bool(re.search('^[0-9-][0-9][\.]+', x))), :]
df3_lon_dirt = df3.loc[~df3['lon'].apply(lambda x: bool(re.search('^[0-9-][0-9][\.]+', x))), :]

df3['lat'] = df3.loc[df3['lat'].apply(lambda x: bool(re.search('^[0-9-][0-9][\.]+', x))), 'lat'].astype(float)
df3['lon'] = df3.loc[df3['lon'].apply(lambda x: bool(re.search('^[0-9-][0-9][\.]+', x))), 'lon'].astype(float)

In [416]:
# total_sqft_dirt = 
df3_total_sqft_dirt = df3.loc[~df3['total_sqft'].apply(lambda x: bool(re.search(r'^([\s\d]+)$', x))), :]

In [417]:
# total_sqft
df3_total_sqft_dirt = df3.loc[~df3['total_sqft'].apply(lambda x: bool(re.search(r'^([\s\d]+)$', x)) ), :]

df3['total_sqft'] = df3.loc[df3['total_sqft'].apply(lambda x: bool(re.search(r'^([\s\d]+)$', x))), 'total_sqft'].astype(int)

## 3.2. Filter Columns

In [418]:
df3.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price,lat,lon,qt_bedroom
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056.0,2.0,1.0,39.07,12.846854,77.676927,2
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600.0,5.0,3.0,120.00,12.895768,77.867101,4
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440.0,2.0,3.0,62.00,12.905568,77.545544,3
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521.0,3.0,1.0,95.00,NaN,NaN,3
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200.0,2.0,1.0,51.00,12.580537,77.333067,2


In [419]:
drop_cols = ['location', 'society', 'size']
df3 = df3.drop(drop_cols, axis=1)
df3.isna().mean()

In [421]:
df3.isna().mean()

area_type       0.000000
availability    0.000000
total_sqft      0.021400
bath            0.000000
balcony         0.000000
price           0.000000
lat             0.112274
lon             0.189142
qt_bedroom      0.000000
dtype: float64

In [422]:
df3 = df3.dropna()
df3.isna().mean()

area_type       0.0
availability    0.0
total_sqft      0.0
bath            0.0
balcony         0.0
price           0.0
lat             0.0
lon             0.0
qt_bedroom      0.0
dtype: float64

In [449]:
round((1 - (df3.shape[0] / df_raw.shape[0])) * 100, 2)

24.53

# 4.0. EDA

In [424]:
df4 = df3.copy()

# 5.0. Data Preparation

In [425]:
df5 = df4.copy()

## 5.1. Standardization

## 5.2. Rescaling

## 5.3. Encoding



In [427]:
le = pp.LabelEncoder()
df5['area_type'] = le.fit_transform(df5[['area_type']])
df5['availability'] = le.fit_transform(df5[['availability']])

/home/cid/.pyenv/versions/3.8.0/envs/local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [428]:
df5.head()

,area_type,availability,total_sqft,bath,balcony,price,lat,lon,qt_bedroom
0,3,37,1056.0,2.0,1.0,39.07,12.846854,77.676927,2
1,2,74,2600.0,5.0,3.0,120.00,12.895768,77.867101,4
2,0,74,1440.0,2.0,3.0,62.00,12.905568,77.545544,3
4,3,74,1200.0,2.0,1.0,51.00,12.580537,77.333067,2
6,3,74,1310.0,3.0,1.0,63.25,12.955257,77.698416,3


# 6.0. Feature Selection

In [437]:
df6 = df5.copy()

In [438]:
X = df5.drop(['price',], axis=1)
y = df5['price'].copy()

x_train, x_val, y_train, y_val = ms.train_test_split(X, y, test_size=0.2, random_state=42)

# 7.0. Model Training

## 7.1. Average Model

In [439]:
# model definition and fit
model_baseline = dummy.DummyRegressor(strategy='mean').fit(x_train, y_train)

# model predict
yhat_baseline = model_baseline.predict(x_val)

# model perfomance
mae = metrics.mean_absolute_error(y_val, yhat_baseline)
mape = metrics.mean_absolute_percentage_error(y_val, yhat_baseline)
rmse = np.sqrt(metrics.mean_squared_error(y_val, yhat_baseline))

print('MAE: {} | MAPE: {} | RMSE: {}'.format(mae, mape, rmse))

MAE: 66.74150511881209 | MAPE: 0.8464271363888621 | RMSE: 134.87690120642173


In [440]:
# MAE: 69.18655009838794 | MAPE: 0.9189057519989557 | RMSE: 130.53405853555498

## 7.2. Linear Regression Model

In [441]:
# model definition and fit
model_lr = lm.LinearRegression().fit(x_train, y_train)

# model predict
yhat_lr = model_lr.predict(x_val)

# model perfomance
mae = metrics.mean_absolute_error(y_val, yhat_lr)
mape = metrics.mean_absolute_percentage_error(y_val, yhat_lr)
rmse = np.sqrt(metrics.mean_squared_error(y_val, yhat_lr))

print('MAE: {} | MAPE: {} | RMSE: {}'.format(mae, mape, rmse))

MAE: 43.14026377555565 | MAPE: 0.4230519670345765 | RMSE: 121.40008663415217


In [433]:
# MAE: 51.39237039425772 | MAPE: 0.5167872441161455 | RMSE: 113.84866787174654

## 7.3. Random Forest Model

In [442]:
# model definition and fit
model_rf = en.RandomForestRegressor().fit(x_train, y_train)

# model predict
yhat_rf = model_rf.predict(x_val)

# model perfomance
mae = metrics.mean_absolute_error(y_val, yhat_rf)
mape = metrics.mean_absolute_percentage_error(y_val, yhat_rf) 
rmse = np.sqrt(metrics.mean_squared_error(y_val, yhat_rf))

print('MAE: {} | MAPE: {} | RMSE: {}'.format(mae, mape, rmse))

MAE: 29.132230679434105 | MAPE: 0.25975983068544 | RMSE: 80.2922239930206


In [435]:
# MAE: 39.84512783025325 | MAPE: 0.357494658863791 | RMSE: 103.6417705071636

# 8.0. Hyperparameter Fine Tuning

# 9.0. Model Perfomance

# 10.0. Deploy to Product